In [9]:
import pandas as pd
import string

In [4]:
df=pd.read_csv('Project11.csv',header=None,sep='|')

In [23]:
df[0][2]

'The third floor contains a promethium generator, a promethium-compatible microchip, a ruthenium generator, and a ruthenium-compatible microchip.'

In [25]:
locdict={1:[],2:[],3:[],4:[]}
elevator_loc=[1]
for i in range(4):
    splitstr=df.iloc[i][0].translate(None,string.punctuation).split(' ')
    for j,word in enumerate(splitstr):
        if word=='microchip' or word=='generator':
            locdict[i+1].append({'element':splitstr[j-1].replace('compatible',''),'type':word})
            

In [26]:
locdict

{1: [{'element': 'thulium', 'type': 'generator'},
  {'element': 'thulium', 'type': 'microchip'},
  {'element': 'plutonium', 'type': 'generator'},
  {'element': 'strontium', 'type': 'generator'}],
 2: [{'element': 'plutonium', 'type': 'microchip'},
  {'element': 'strontium', 'type': 'microchip'}],
 3: [{'element': 'promethium', 'type': 'generator'},
  {'element': 'promethium', 'type': 'microchip'},
  {'element': 'ruthenium', 'type': 'generator'},
  {'element': 'ruthenium', 'type': 'microchip'}]}

In [19]:
'asdf'.replace('df','')

'as'

In [31]:
df1=pd.DataFrame(locdict[1])
df1['floor']=1
df2=pd.DataFrame(locdict[2])
df2['floor']=2
df3=pd.DataFrame(locdict[3])
df3['floor']=3

finaldf=pd.concat([df1,df2,df3]).reset_index(drop=True)

In [33]:
finaldf

,element,type,floor
0,thulium,generator,1
1,thulium,microchip,1
2,plutonium,generator,1
3,strontium,generator,1
4,plutonium,microchip,2
5,strontium,microchip,2
6,promethium,generator,3
7,promethium,microchip,3
8,ruthenium,generator,3
9,ruthenium,microchip,3


In [48]:
def make_map(s):
    colname=s.element+'_'+s['type']
    newseries=pd.Series([colname,1],index=['colname',s.floor],name=colname)
    
    return newseries

finaldf_alt = finaldf.apply(make_map,axis=1).set_index('colname').transpose().fillna(0)
finaldf_alt.loc[4]=0
finaldf_alt.sort_index(ascending=False,inplace=True)

In [53]:
finaldf_alt.style.highlight_max(color='green')

In [ ]:
# need to do optimization type algorithm
# first identify number of valid moves from the current floor (1 or 2 machines)
# start a chain from each branch, which will recursively call the identification function
# invalid moves will also be passed, which is the reverse of what just happened (don't move machine up, then back down)
# check to see if all machiens are on the top floor: if so, end chain, and list of moves
# look at all chains, and find one with min length

In [171]:
finaldf

,element,type,floor
0,thulium,generator,1
1,thulium,microchip,1
2,plutonium,generator,1
3,strontium,generator,1
4,plutonium,microchip,2
5,strontium,microchip,2
6,promethium,generator,3
7,promethium,microchip,3
8,ruthenium,generator,3
9,ruthenium,microchip,3


In [174]:
pd.DataFrame([x.split('_') for x in finaldf_alt.loc[1].index])

,0,1
0,thulium,generator
1,thulium,microchip
2,plutonium,generator
3,strontium,generator
4,plutonium,microchip
5,strontium,microchip
6,promethium,generator
7,promethium,microchip
8,ruthenium,generator
9,ruthenium,microchip


In [128]:
currentdf=finaldf_alt.copy(deep=True)
currfloor=currentdf.loc[1]
currfloor=currfloor[currfloor==1]

In [142]:
comblist=currfloor.index.tolist()
comblist2=[[x] for x in comblist]
comblist2

[['thulium_generator'],
 ['thulium_microchip'],
 ['plutonium_generator'],
 ['strontium_generator']]

In [144]:
[list(x) for x in itertools.combinations(comblist,2)]

[['thulium_generator', 'thulium_microchip'],
 ['thulium_generator', 'plutonium_generator'],
 ['thulium_generator', 'strontium_generator'],
 ['thulium_microchip', 'plutonium_generator'],
 ['thulium_microchip', 'strontium_generator'],
 ['plutonium_generator', 'strontium_generator']]

In [181]:
def move_items(currentdf,move,elv_loc_prev):
    if move[0]=='up':
        movedir=1
    elif move[0]=='down':
        movedir=-1
    else:
        raise ValueError
        
    
    

def check_floors(currentdf):
    for i in range(1,5):
        floor=currentdf.loc[i]
        floor=floor[floor==1]
        if len(floor)>0:
            tempdf= pd.DataFrame([x.split('_') for x in floor.index])
            for element in tempdf[0].unique():
                subdf=tempdf[tempdf[0]==element]
                if len(subdf)==2:
                    tempdf = tempdf[tempdf[0]!=element]
            if len(tempdf[tempdf[1]=='generator'])>0 and len(tempdf[tempdf[1]=='microchip'])>0:
                return False
        
    return True
        
        # remove pairs
        

def loc_possible_moves(currentdf,previousmove,elv_loc):
    #first get all possible combinations from the current floor
    currfloor=currentdf.loc[elv_loc]
    currfloor=currfloor[currfloor==1].sort_index()
    comblist=currfloor.index.tolist()
    comblist2=[[x] for x in comblist]
    print currfloor.index.tolist()
    print comblist
    for x in itertools.combinations(comblist,2):
        print x
    comblist2+=[list(x) for x in itertools.combinations(comblist,2)]
    
    if elv_loc==1:
        comblist2=[['up']+x for x in comblist2]
    elif elv_loc==4:
        comblist2=[['down']+x for x in comblist2]
    else:
        comblist2=[['down']+x for x in comblist2]+[['up']+x for x in comblist2]
    if len(previousmove)>0:
        if previousmove[0]=='up':
            previousmove[0]=='down'
        else:
            previousmove[0]=='up'
        comblist.remove(previousmove)
    
    return comblist2


def next_stop(currentdf,previousmove,elv_loc):
    
    out = check_floors(currentdf)
    
    possible_moves = loc_possible_moves(currentdf,[],elv_loc)

In [179]:
check_floors(finaldf_alt)

True

In [170]:
loc_possible_moves(finaldf_alt,[],2)


['plutonium_microchip', 'strontium_microchip']
['plutonium_microchip', 'strontium_microchip']
('plutonium_microchip', 'strontium_microchip')


[['down', 'plutonium_microchip'],
 ['down', 'strontium_microchip'],
 ['down', 'plutonium_microchip', 'strontium_microchip'],
 ['up', 'plutonium_microchip'],
 ['up', 'strontium_microchip'],
 ['up', 'plutonium_microchip', 'strontium_microchip']]

In [162]:
['up', 'plutonium_microchip'] in [gg]

True

In [166]:
gg.remove('up')

ValueError: list.remove(x): x not in list

In [165]:
gg

['plutonium_microchip']

In [57]:
import itertools

In [125]:
qqq=[list(x) for x in itertools.combinations(['a','b','c'],2)]

In [126]:
qqq

[['a', 'b'], ['a', 'c'], ['b', 'c']]

In [75]:
f=['a','b','c']

In [76]:
f.insert(0,'d')

In [77]:
f

['d', 'a', 'b', 'c']

In [185]:
raise ValueError

ValueError: 